In [1]:
import subprocess
import tensorflow as tf
import optuna
import os

# Create shared SQLite study
study_name = "multi_gpu_optuna_study"
storage_url = "sqlite:///optuna_multi_gpu.db"

try:
    optuna.create_study(study_name=study_name, direction='maximize', storage=storage_url)
except optuna.exceptions.DuplicatedStudyError:
    pass

# Detect available GPUs
gpus = tf.config.list_physical_devices('GPU')
n_trials_per_gpu = 25  # Or more

# Spawn one subprocess per GPU
processes = []
for gpu_id in range(len(gpus)):
    cmd = [
        "python", "optuna_worker.py",
        str(gpu_id),
        str(n_trials_per_gpu),
        study_name,
        storage_url
    ]
    print("Launching:", " ".join(cmd))
    p = subprocess.Popen(cmd)
    processes.append(p)

# Wait for all workers to finish
for p in processes:
    p.wait()


Launching: python optuna_worker.py 0 25 multi_gpu_optuna_study sqlite:///optuna_multi_gpu.db


2025-04-01 22:56:05.179291: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-04-01 22:56:05.179318: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-01 22:56:05.179324: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-01 22:56:05.179338: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-01 22:56:05.179346: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-04-01 22:56:06.135145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
[W 2025-04-01 22:56:15,938] Tri

KeyboardInterrupt: 